### Python DataWrangling Pandas Combining

_Combine DataFrames with .concat()_

In this lesson, we'll return to the video game sales dataset. Here are the first few rows to remind you of its structure:

In [7]:
import pandas as pd

df = pd.read_csv('DataSets/vg_sales.csv')
print(df.head())

                       name platform  year_of_release         genre publisher  \
0                Wii Sports      Wii           2006.0        Sports  Nintendo   
1         Super Mario Bros.      NES           1985.0      Platform  Nintendo   
2            Mario Kart Wii      Wii           2008.0        Racing  Nintendo   
3         Wii Sports Resort      Wii           2009.0        Sports  Nintendo   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

  developer  na_sales  eu_sales  jp_sales  critic_score user_score  
0  Nintendo     41.36     28.96      3.77          76.0          8  
1       NaN     29.08      3.58      6.81           NaN        NaN  
2  Nintendo     15.68     12.76      3.79          82.0        8.3  
3  Nintendo     15.61     10.93      3.28          80.0          8  
4       NaN     11.27      8.89     10.22           NaN        NaN  


We want to know some general statistics about game publishers, specifically:

- their average review score;
- their total sales.

As we've already seen, we can do this using groupby(). First, let's get the average review score for each publisher:

In [8]:
mean_score = df.groupby('publisher')['critic_score'].mean()
print(mean_score)

publisher
10TACLE Studios                 42.000000
1C Company                      73.000000
20th Century Fox Video Games          NaN
2D Boy                          90.000000
3DO                             57.470588
                                  ...    
id Software                     85.000000
imageepoch Inc.                       NaN
inXile Entertainment            81.000000
mixi, Inc                             NaN
responDESIGN                          NaN
Name: critic_score, Length: 581, dtype: float64


Let's also get the number of sales. The easiest way to do this is with a second groupby():

In [9]:
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']
num_sales = df.groupby('publisher')['total_sales'].sum()
print(num_sales)

publisher
10TACLE Studios                 0.11
1C Company                      0.08
20th Century Fox Video Games    1.92
2D Boy                          0.03
3DO                             9.52
                                ... 
id Software                     0.02
imageepoch Inc.                 0.04
inXile Entertainment            0.09
mixi, Inc                       0.87
responDESIGN                    0.13
Name: total_sales, Length: 581, dtype: float64


Notice that the index for both results is the 'publisher' column _because we grouped by 'publisher' in both cases_. __Since both results share the same index, we can easily join the results__ into a DataFrame using pandas' concat() function:

In [10]:
df_concat = pd.concat([mean_score, num_sales], axis='columns')
print(df_concat)

                              critic_score  total_sales
publisher                                              
10TACLE Studios                  42.000000         0.11
1C Company                       73.000000         0.08
20th Century Fox Video Games           NaN         1.92
2D Boy                           90.000000         0.03
3DO                              57.470588         9.52
...                                    ...          ...
id Software                      85.000000         0.02
imageepoch Inc.                        NaN         0.04
inXile Entertainment             81.000000         0.09
mixi, Inc                              NaN         0.87
responDESIGN                           NaN         0.13

[581 rows x 2 columns]


In general, concat() expects a list of Series and/or DataFrame objects. To get our result, we passed a list of Series variables to concat() and set axis='columns' to ensure they were combined as columns.

Note that the original column names are preserved in the concatenated DataFrame.

We can rename columns using the columns method. It can be called on a DataFrame and passed a list of new column names to replace the existing ones. The new names must be passed in the same order as the original column names.

Let's rename 'critic_score', as it now represents an average:

In [11]:
df_concat.columns = ['avg_critic_score', 'total_sales']
print(df_concat)

                              avg_critic_score  total_sales
publisher                                                  
10TACLE Studios                      42.000000         0.11
1C Company                           73.000000         0.08
20th Century Fox Video Games               NaN         1.92
2D Boy                               90.000000         0.03
3DO                                  57.470588         9.52
...                                        ...          ...
id Software                          85.000000         0.02
imageepoch Inc.                            NaN         0.04
inXile Entertainment                 81.000000         0.09
mixi, Inc                                  NaN         0.87
responDESIGN                               NaN         0.13

[581 rows x 2 columns]


In general, it's a good idea to rename columns after grouping and processing to give a more indicative representation of how the columns were processed.

You may have noticed that we could get the same result as before using agg(). However, concat() is quite versatile. We can use it to concatenate DataFrames:

- by rows, assuming they have the same number of columns;

- by columns if they have the same number of rows.

To concatenate rows from separate DataFrames, we can use concat() and set axis='index' (or omit this parameter, as axis='index' is the default argument). Alternatively, we can use integers for the index= argument, where index=0 will concatenate rows and index=1 will concatenate columns.

Here's an example where we filter the data in two separate DataFrames based on gender and then recombine them into a single DataFrame:

In [12]:
rpgs = df[df['genre'] == 'Role-Playing']
platformers = df[df['genre'] == 'Platform']

df_concat = pd.concat([rpgs, platformers])
print(df_concat[['name', 'genre']])

                                                   name         genre
4                              Pokemon Red/Pokemon Blue  Role-Playing
12                          Pokemon Gold/Pokemon Silver  Role-Playing
20                        Pokemon Diamond/Pokemon Pearl  Role-Playing
25                        Pokemon Ruby/Pokemon Sapphire  Role-Playing
27                          Pokemon Black/Pokemon White  Role-Playing
...                                                 ...           ...
16356                                    Strider (2014)      Platform
16358                                Goku Makaimura Kai      Platform
16603  The Land Before Time: Into the Mysterious Beyond      Platform
16710                Woody Woodpecker in Crazy Castle 5      Platform
16715                                  Spirits & Spells      Platform

[2388 rows x 2 columns]


And so two DataFrames are merged into one! Remember, __this works here because both smaller DataFrames have the same columns__.

##### __Excercise 01__

We read the data, created a 'total_sales' column, and calculated the total sales for each platform in the total_sales variable.

You need to calculate the total number of publishers that created a game on each platform using nunique(). Assign the result to a variable called num_pubs and then display it.

In [13]:
import pandas as pd

df = pd.read_csv('DataSets/vg_sales.csv')
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']

total_sales = df.groupby('platform')['total_sales'].sum()

num_pubs =  df.groupby('platform')['publisher'].nunique()
print(total_sales)
print()
print(num_pubs)

platform
2600      96.07
3DO        0.10
3DS      245.64
DC        15.68
DS       747.13
GB       247.26
GBA      310.12
GC       193.75
GEN       27.46
GG         0.04
N64      214.30
NES      245.74
NG         1.44
PC       237.14
PCFX       0.03
PS       689.95
PS2     1062.33
PS3      803.97
PS4      265.83
PSP      252.63
PSV       47.63
SAT       33.52
SCD        1.81
SNES     196.82
TG16       0.16
WS         1.42
Wii      828.44
WiiU      76.24
X360     885.66
XB       249.02
XOne     145.05
Name: total_sales, dtype: float64

platform
2600     26
3DO       3
3DS      82
DC       15
DS      175
GB       17
GBA      87
GC       52
GEN       7
GG        1
N64      54
NES      12
NG        3
PC      133
PCFX      1
PS      151
PS2     172
PS3     103
PS4      75
PSP     127
PSV      66
SAT      44
SCD       1
SNES     50
TG16      2
WS        2
Wii     113
WiiU     23
X360    102
XB       73
XOne     46
Name: publisher, dtype: int64


Combine total_sales and num_pubs columns into a DataFrame named platforms using concat(). Change the column names in platforms to 'total_sales' and 'num_publishers', respectively, then print platforms.

In [14]:
platforms = pd.concat([total_sales, num_pubs], axis='columns')
platforms.columns = ['total_sales', 'num_publishers']
print(platforms[['total_sales', 'num_publishers']])

          total_sales  num_publishers
platform                             
2600            96.07              26
3DO              0.10               3
3DS            245.64              82
DC              15.68              15
DS             747.13             175
GB             247.26              17
GBA            310.12              87
GC             193.75              52
GEN             27.46               7
GG               0.04               1
N64            214.30              54
NES            245.74              12
NG               1.44               3
PC             237.14             133
PCFX             0.03               1
PS             689.95             151
PS2           1062.33             172
PS3            803.97             103
PS4            265.83              75
PSP            252.63             127
PSV             47.63              66
SAT             33.52              44
SCD              1.81               1
SNES           196.82              50
TG16        

##### _Merging DataFrames with merge()_

You've just learned how to combine DataFrames by concatenating them by row or column using concat().

Concatenating DataFrames preserves the total amount of data. For example, merging a DataFrame that has two columns and three rows with another DataFrame that has the same two columns and five rows results in a DataFrame with two columns and eight rows. The total number of cells before and after concatenation is sixteen.

Now you'll learn how to merge DataFrames using the merge() method in a way that affects the amount of data you're working with.

Consider the following example: two literature students agree that one will write half of the summer reading list on the board while the other watches YouTube. Then, the first will go to the cafeteria, while the second copies the rest of the list. Finally, the two will merge the lists. Teamwork! Let's see how it went:

In [15]:
import pandas as pd

first_pupil_df = pd.DataFrame(
    {
        'author': ['Alcott', 'Fitzgerald', 'Steinbeck', 'Twain', 'Hemingway'],
        'title': ['Little Women',
                  'The Great Gatsby',
                  'Of Mice and Men',
                  'The Adventures of Tom Sawyer',
                  'The Old Man and the Sea'
                 ],
    }
)
second_pupil_df = pd.DataFrame(
    {
        'author': ['Steinbeck', 'Twain', 'Hemingway', 'Salinger', 'Hawthorne'],
        'title': ['East of Eden',
                  'The Adventures of Huckleberry Finn',
                  'For Whom the Bell Tolls',
                  'The Catcher in the Rye',
                  'The Scarlett Letter'
                 ],
    }
)

print(first_pupil_df)
print()
print(second_pupil_df)

       author                         title
0      Alcott                  Little Women
1  Fitzgerald              The Great Gatsby
2   Steinbeck               Of Mice and Men
3       Twain  The Adventures of Tom Sawyer
4   Hemingway       The Old Man and the Sea

      author                               title
0  Steinbeck                        East of Eden
1      Twain  The Adventures of Huckleberry Finn
2  Hemingway             For Whom the Bell Tolls
3   Salinger              The Catcher in the Rye
4  Hawthorne                 The Scarlett Letter


##### _Inner Join_

Let's use the merge() method to combine entries with the same authors. The name of the column to be merged is passed to the on= parameter, in this case, 'author':

In [16]:
both_pupils = first_pupil_df.merge(second_pupil_df, on='author')
print(both_pupils) 

      author                       title_x                             title_y
0  Steinbeck               Of Mice and Men                        East of Eden
1      Twain  The Adventures of Tom Sawyer  The Adventures of Huckleberry Finn
2  Hemingway       The Old Man and the Sea             For Whom the Bell Tolls


The result contains only those authors present in both original DataFrames.

The merged DataFrame includes all columns from the original DataFrames, but only the rows with shared authors are retained. Since both original DataFrames have a column named 'title', pandas added the suffixes _x and _y to differentiate them in the merged DataFrame. It's worth noting that the merged DataFrame only has 9 cells, compared to 20 cells in the original DataFrames: the amount of data has changed!

This way of merging is called an inner merge. There are other types of merges, which can be specified with the how= parameter of merge(). But 'inner' is the default argument to how=, so we don't need to include it above.

##### _Outer join_

An outer join (merge) differs from an inner join in that all values ​​in the specified column are retained from both original DataFrames, but the merged DataFrame has missing values ​​where there is no match. This is best illustrated with an example:

In [17]:
both_pupils = first_pupil_df.merge(second_pupil_df, on='author', how='outer')
print(both_pupils)

       author                       title_x  \
0      Alcott                  Little Women   
1  Fitzgerald              The Great Gatsby   
2   Hawthorne                           NaN   
3   Hemingway       The Old Man and the Sea   
4    Salinger                           NaN   
5   Steinbeck               Of Mice and Men   
6       Twain  The Adventures of Tom Sawyer   

                              title_y  
0                                 NaN  
1                                 NaN  
2                 The Scarlett Letter  
3             For Whom the Bell Tolls  
4              The Catcher in the Rye  
5                        East of Eden  
6  The Adventures of Huckleberry Finn  


There are 7 unique authors in both original DataFrames, each represented by a row in the merged DataFrame. For authors in the first DataFrame who are not also in the second (i.e., 'Alcott' and 'Fitzgerald'), there are NaN values ​​in the column that comes from the second DataFrame (i.e., 'title_y'), and vice versa. Also, notice that we now have 21 data cells.

##### _Left join_

The last type of join we'd like to discuss is the left join (left merge), which we can perform by passing how='left' to merge(). In a left join, all values ​​from the left DataFrame (the one we call merge() on) are present in the merged DataFrame. Values ​​from the right DataFrame (the one we pass as input to merge()) are only retained for values ​​that match the specified column in the left DataFrame. Again, this is best explained with an example:

In [18]:
both_pupils = first_pupil_df.merge(second_pupil_df, on='author', how='left')
print(both_pupils)

       author                       title_x  \
0      Alcott                  Little Women   
1  Fitzgerald              The Great Gatsby   
2   Steinbeck               Of Mice and Men   
3       Twain  The Adventures of Tom Sawyer   
4   Hemingway       The Old Man and the Sea   

                              title_y  
0                                 NaN  
1                                 NaN  
2                        East of Eden  
3  The Adventures of Huckleberry Finn  
4             For Whom the Bell Tolls  


As you can see, all of the first student's authors and titles are in the merged DataFrame, but the second student's rows with 'Salinger' and 'Hawthorne' are not because those authors don't appear in the first student's DataFrame.

This left join contains 15 cells of data, which differ from the original number and amounts in each of the other joins we performed.

Note that there is also a right join (how='right'). However, it works identically to a left join, except that the merged DataFrame retains all the values ​​from the right DataFrame instead of the left. The same result can be achieved by performing a left join and changing the order of the DataFrames.

Here's a Venn diagram illustrating all the merge options we've discussed, to make it even easier to understand:

![Alt Text](Pics/merge.png)

Consider the Column Names

There are two aspects of all the joins performed so far that we need to address:

- The merged DataFrame has the suffixes _x and _y appended to the 'title' column names.

- The column we are joining on has the same name in both DataFrames, 'author'.

When merging DataFrames in pandas, it is important to ensure that no two columns have the same name. Otherwise, pandas will automatically add the suffixes _x and _y. However, these suffixes are not very descriptive. To set better suffixes, pass a tuple of suffixes to the suffixes= parameter in merge():

In [19]:
both_pupils = first_pupil_df.merge(second_pupil_df, on='author', suffixes=('_1st_student', '_2nd_student')
                                  )
print(both_pupils)

      author             title_1st_student                   title_2nd_student
0  Steinbeck               Of Mice and Men                        East of Eden
1      Twain  The Adventures of Tom Sawyer  The Adventures of Huckleberry Finn
2  Hemingway       The Old Man and the Sea             For Whom the Bell Tolls


##### _Excercise 01_

We have two DataFrames, df_orders and df_members, which have already been included in the precode.

- df_orders:
* Each row represents a single order.
* Contains a column named 'user_id' that indicates which customer placed the order.
* Contains an 'id' column that identifies each order.

- df_members:
* Each row represents a single customer.
* Contains an 'id' column that identifies each customer.

Your task:

Merge these two DataFrames so that the resulting DataFrame includes only those customers who have placed an order. Follow these steps:

- Merge type:
* Choose a merge that retains only customers with matching orders (for example, an inner merge).
- Merge details:
* Use df_members as the left DataFrame and match the customer id.
* Use df_orders as the right DataFrame and match the 'user_id' column (not the order_id).
- Column Suffixes:
* Apply suffixes to overlapping column names: add '_member' to the columns in df_members and '_order' to the columns in df_orders.
- Store and Display:
* Assign the merged DataFrame to a variable named df_merged.
* Display df_merged to review the results.

Note:
Do not delete any columns during the merge.

In [22]:
import pandas as pd

df_members = pd.read_csv('DataSets/new_members.csv')
df_orders  = pd.read_csv('DataSets/recent_orders.csv')

print(df_members)
print()
print(df_orders)
print()

df_merged = df_members.merge(df_orders, left_on='id', right_on='user_id', how='inner', suffixes=('_members', '_orders'))

print(df_merged)

     id       username date_joined
0  9832  monkeycool447  2021-06-19
1  9833      Appl3Girl  2021-06-19
2  9834       hiiml00t  2021-06-19
3  9835       jd663366  2021-06-19
4  9836   watermelon89  2021-06-19
5  9837        SUPERXD  2021-06-19
6  9838    aasmith0909  2021-06-19
7  9839     NotHotDog2  2021-06-20
8  9840       starrats  2021-06-20
9  9841      beat1box2  2021-06-20

           id  user_id    service_id      order_timestamp
0   163548290     9836  XMD8nVShpINn  2021-06-22Z18:32:59
1   163548291     9836  PXAQ9MiP7BvW  2021-06-22Z18:32:59
2   163548292     8725  ikNjwXadFlDy  2021-06-22Z18:33:19
3   163548293     9840  9KyrlovWf2nH  2021-06-22Z18:34:00
4   163548294     9121  WXoCdFvGgXmb  2021-06-22Z18:35:16
5   163548295     9121  idVIMieuoAMU  2021-06-22Z18:35:16
6   163548296     9121  CC4F1NgnYGs3  2021-06-22Z18:35:16
7   163548297     9839  NvwWjzW7FydE  2021-06-22Z18:36:21
8   163548298     9841  fCobsButtJD7  2021-06-22Z18:36:55
9   163548299     9837  R2GA1xIVXK

Let's tidy things up a bit.

- Remove the duplicate column (in this case, 'user_id').
- Assign the result back to the 'df_merged' DataFrame.
- Display the merged DataFrame.

In [21]:
df_merged = df_merged.drop(labels='user_id', axis='columns') 
print(df_merged)

   id_members      username date_joined  id_orders    service_id  \
0        9836  watermelon89  2021-06-19  163548290  XMD8nVShpINn   
1        9836  watermelon89  2021-06-19  163548291  PXAQ9MiP7BvW   
2        9837       SUPERXD  2021-06-19  163548299  R2GA1xIVXK1o   
3        9839    NotHotDog2  2021-06-20  163548297  NvwWjzW7FydE   
4        9840      starrats  2021-06-20  163548293  9KyrlovWf2nH   
5        9841     beat1box2  2021-06-20  163548298  fCobsButtJD7   

       order_timestamp  
0  2021-06-22Z18:32:59  
1  2021-06-22Z18:32:59  
2  2021-06-22Z18:39:12  
3  2021-06-22Z18:36:21  
4  2021-06-22Z18:34:00  
5  2021-06-22Z18:36:55  
